In [1]:
import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns 
import yfinance as yf
from datetime import datetime, timedelta
import backtrader as bt


In [2]:
plt.style.use('seaborn-v0_8')  
plt.rcParams['figure.figsize'] = (16, 8)  
plt.rcParams['font.size'] = 14  

In [ ]:
ticker_symbol = 'GRRR'

start_date = '2022-3-27'  
end_date = '2025-3-27'

stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
print(f"The data is succesfully extracted for stock {ticker_symbol}, from{ start_date} to {end_date}，in total of {len(stock_data)} entries") 
stock_data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

The data is succesfully extracted for stock GRRR, from2022-3-27 to 2025-3-27，in total of 752 entries


Price,Close,High,Low,Open,Volume
Ticker,GRRR,GRRR,GRRR,GRRR,GRRR
Date,,,,,
2022-03-28,100.949997,101.000000,100.949997,101.0,810
2022-03-29,101.000000,101.099998,101.000000,101.0,7820
2022-03-30,101.000000,101.000000,101.000000,101.0,280
2022-03-31,100.900002,101.000000,100.900002,101.0,3240
2022-04-01,101.000000,101.000000,101.000000,101.0,220


In [4]:
df = stock_data.copy()
df.columns = stock_data.columns.droplevel(0) 
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(inplace=True)
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2022-03-28,100.949997,101.000000,100.949997,101.0,810
2022-03-29,101.000000,101.099998,101.000000,101.0,7820
2022-03-30,101.000000,101.000000,101.000000,101.0,280
2022-03-31,100.900002,101.000000,100.900002,101.0,3240
2022-04-01,101.000000,101.000000,101.000000,101.0,220


In [5]:
csv_file = f"data/{ticker_symbol}_stock_data.csv"
df.to_csv(csv_file)

In [6]:
def create_bt_data_feed(csv_file):  #
    return bt.feeds.YahooFinanceCSVData(  
        dataname=csv_file, 
        fromdate=datetime.strptime(start_date, '%Y-%m-%d'), 
        todate=datetime.strptime(end_date, '%Y-%m-%d'),
        reverse=False  
    )

In [7]:
def run_backtest(strategy_class, data_feed, cash=100000.0, commission=0.001, **strategy_params):
    
    # Initialize Backtrader engine
    cerebro = bt.Cerebro()
    
    # Add data feed
    cerebro.adddata(data_feed)
    
    # Add strategy
    cerebro.addstrategy(strategy_class, **strategy_params)
    
    # Set initial capital and commission
    cerebro.broker.setcash(cash)
    cerebro.broker.setcommission(commission=commission)
    
    # Add performance analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    
    # Run backtest
    initial_value = cerebro.broker.getvalue()
    print(f'Initial Capital: ${initial_value:.2f}')
    
    results = cerebro.run()
    strategy = results[0]
    
    
    return cerebro, strategy, results

In [16]:
class MovingAverageCrossOverStrategy(bt.Strategy):
    params = (  
        ('short_period', 20), 
        ('long_period', 50),  
        ('printlog', True),  
    )

    def log(self, txt, dt=None):
        if self.params.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')
    
    def __init__(self):
        self.order = None
        self.short_ma = bt.indicators.SimpleMovingAverage(self.datas[0].close, period=self.params.short_period)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.datas[0].close, period=self.params.long_period)
        self.crossover = bt.indicators.CrossOver(self.short_ma, self.long_ma)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status == order.Completed:
            if order.isbuy():
                self.log(f'BUY EXECUTED: Price: {order.executed.price:.2f}, Size: {order.executed.size}')
            else:
                self.log(f'SELL EXECUTED: Price: {order.executed.price:.2f}, Size: {order.executed.size}')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('ORDER CANCELED/REJECTED')
        
        self.order = None

    def next(self):
        if self.order:
            return

        if not self.position:
            if self.crossover > 0:
                cash = self.broker.getcash()
                price = self.datas[0].close[0]
                size = int(cash // price)
                if size > 0:
                    self.log(f'BUY SIGNAL: Buying {size} shares')
                    self.order = self.buy(size=size)
        else:
            if self.crossover < 0:
                self.log(f'SELL SIGNAL: Selling all {self.position.size} shares')
                self.order = self.sell(size=self.position.size)
    def stop(self):
        self.log(f'Strategy Ended: Final Portfolio Value: {self.broker.getvalue():.2f}')

In [17]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-03-28,100.949997,101.000000,100.949997,101.000000,810
2022-03-29,101.000000,101.099998,101.000000,101.000000,7820
2022-03-30,101.000000,101.000000,101.000000,101.000000,280
2022-03-31,100.900002,101.000000,100.900002,101.000000,3240
2022-04-01,101.000000,101.000000,101.000000,101.000000,220
...,...,...,...,...,...
2025-03-20,23.670000,26.389999,22.700001,26.170000,3933000
2025-03-21,25.150000,25.750000,22.299999,22.370001,2546800
2025-03-24,27.219999,28.200001,25.389999,28.120001,3446600


In [18]:
cerebro = bt.Cerebro() 
cerebro.broker.setcash(100000.0)  

cerebro.addstrategy(MovingAverageCrossOverStrategy, short_period=20, long_period=50, printlog=True)

data_feed = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data_feed)

print('Initial Investment: %.2f' % cerebro.broker.getvalue())

cerebro.run()  

print('Final value: %.2f' % cerebro.broker.getvalue()) 

Initial Investment: 100000.00
2023-02-27, BUY SIGNAL: Buying 1427 shares
2023-02-28, ORDER CANCELED/REJECTED
2023-03-15, BUY SIGNAL: Buying 1248 shares
2023-03-16, ORDER CANCELED/REJECTED
2023-06-30, BUY SIGNAL: Buying 4784 shares
2023-07-03, ORDER CANCELED/REJECTED
2024-02-07, BUY SIGNAL: Buying 8620 shares
2024-02-08, BUY EXECUTED: Price: 9.30, Size: 8620
2024-04-02, SELL SIGNAL: Selling all 8620 shares
2024-04-03, SELL EXECUTED: Price: 7.32, Size: -8620
2024-09-06, BUY SIGNAL: Buying 25361 shares
2024-09-09, BUY EXECUTED: Price: 3.21, Size: 25361
2025-03-26, Strategy Ended: Final Portfolio Value: 674604.55
Final value: 674604.55
